<a href="https://colab.research.google.com/github/wagner1986/PapyrusTech/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from abc import ABC, abstractmethod
from typing import Any, List, Tuple
from itertools import product

class IPreprocessor(ABC):
    @abstractmethod
    def preprocess(self, image: Any) -> Any: pass

class IDetector(ABC):
    @abstractmethod
    def detect(self, image: Any) -> Tuple[List[Any], List[Any]]: pass

class ILocalMatcher(ABC):
    @abstractmethod
    def match(self, descriptors1: List[Any], descriptors2: List[Any]) -> List[Any]: pass

class IGlobalStructurer(ABC):
    @abstractmethod
    def structure(self, keypoints: List[Any]) -> Any: pass

class IGlobalMatcher(ABC):
    @abstractmethod
    def match_global(self, structure1: Any, structure2: Any) -> float: pass

class ImageComparisonPipeline:
    __slots__ = ['preprocessor', 'detector', 'local_matcher', 'global_structurer', 'global_matcher']

    def __init__(self, preprocessor: IPreprocessor = None, detector: IDetector = None, local_matcher: ILocalMatcher = None, global_structurer: IGlobalStructurer = None, global_matcher: IGlobalMatcher = None):
        self.preprocessor = preprocessor
        self.detector = detector
        self.local_matcher = local_matcher
        self.global_structurer = global_structurer
        self.global_matcher = global_matcher

    def run(self, image1: Any, image2: Any) -> float:
        if not all([self.preprocessor, self.detector, self.local_matcher, self.global_structurer, self.global_matcher]):
            raise ValueError("Pipeline components are not fully set.")
        image1_processed, image2_processed = map(self.preprocessor.preprocess, (image1, image2))
        keypoints1, descriptors1 = self.detector.detect(image1_processed)
        keypoints2, descriptors2 = self.detector.detect(image2_processed)
        matches = self.local_matcher.match(descriptors1, descriptors2)
        matched_keypoints1 = [keypoints1[match.queryIdx] for match in matches]
        matched_keypoints2 = [keypoints2[match.trainIdx] for match in matches]
        structured_kp1 = self.global_structurer.structure(matched_keypoints1)
        structured_kp2 = self.global_structurer.structure(matched_keypoints2)
        return self.global_matcher.match_global(structured_kp1, structured_kp2)

class PipelineGridSearch:
    def __init__(self, pipeline: ImageComparisonPipeline, configurations: dict):
        self.pipeline = pipeline
        self.configurations = configurations

    def search(self, image1: Any, image2: Any) -> Tuple[float, dict]:
        best_score, best_configuration = -float('inf'), {}

        # Correção aqui: Alterar para capturar a configuração atual corretamente
        for configuration in product(*self.configurations.values()):
            preprocess, detector, local_matcher, global_structurer, global_matcher = configuration
            self.pipeline.preprocessor = preprocess
            self.pipeline.detector = detector
            self.pipeline.local_matcher = local_matcher
            self.pipeline.global_structurer = global_structurer
            self.pipeline.global_matcher = global_matcher

            try:
                score = self.pipeline.run(image1, image2)
                if score > best_score:
                    best_score = score
                    # Use a variável 'configuration' em vez de 'config'
                    best_configuration = dict(zip(self.configurations.keys(), configuration))
            except Exception as e:
                # Use a variável 'configuration' para imprimir em caso de erro
                print(f"Error with configuration: {configuration}. Error: {e}")

        return best_score, best_configuration



In [29]:
class PreprocessorMock(IPreprocessor):
    def __init__(self, name):
        self.name = name

    def preprocess(self, image):
        return image  # Em uma implementação real, o processamento seria feito aqui

    def __str__(self):
        return f"Preprocessing with {self.name}"

class DetectorMock(IDetector):
    def __init__(self, name):
        self.name = name

    def detect(self, image):
        return [], []  # Em uma implementação real, os keypoints e descriptors seriam retornados
    def __str__(self):
        return f"Detecting with {self.name}"

class LocalMatcherMock(ILocalMatcher):
    def __init__(self, name):
        self.name = name

    def match(self, descriptors1, descriptors2):
        return []  # Em uma implementação real, os matches seriam retornados
    def __str__(self):
        return f"Local matching with {self.name}"

class GlobalStructurerMock(IGlobalStructurer):
    def __init__(self, name):
        self.name = name

    def structure(self, keypoints):

        return []  # Em uma implementação real, as correspondências estruturadas seriam retornadas
    def __str__(self):
        return f"Global structuring with {self.name}"

class GlobalMatcherMock(IGlobalMatcher):
    def __init__(self, name):
        self.name = name

    def match_global(self, structure1,structure2):
        return 0.9  # Em uma implementação real, a pontuação de similaridade seria calculada

    def __str__(self):
        return f"Global matching with {self.name}"


In [30]:
# Configuração das implementações mock
preprocessors = [PreprocessorMock("Preprocessor A"), PreprocessorMock("Preprocessor B")]
detectors = [DetectorMock("Detector A"), DetectorMock("Detector B")]
local_matchers = [LocalMatcherMock("Local Matcher A"), LocalMatcherMock("Local Matcher B")]
global_structurers = [GlobalStructurerMock("Global Structurer A"), GlobalStructurerMock("Global Structurer B")]
global_matchers = [GlobalMatcherMock("Global Matcher A"), GlobalMatcherMock("Global Matcher B")]

configurations = {
    'preprocessor': preprocessors,
    'detector': detectors,
    'local_matcher': local_matchers,
    'global_structurer': global_structurers,
    'global_matcher': global_matchers
}

# Inicialização do pipeline e do grid search
pipeline = ImageComparisonPipeline()
grid_search = PipelineGridSearch(pipeline, configurations)

# Execução do grid search
best_score, best_configuration = grid_search.search("image1", "image2")

print(f"Best Score: {best_score}")
print("Best Configuration:")
for key, strategy in best_configuration.items():
    print(f"{key}: {strategy.name}")


Best Score: 0.9
Best Configuration:
preprocessor: Preprocessor A
detector: Detector A
local_matcher: Local Matcher A
global_structurer: Global Structurer A
global_matcher: Global Matcher A
